In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('drug_data.csv')
le = LabelEncoder()
df['approval_status_encoded'] = le.fit_transform(df['approval_status'])


In [2]:
df = df.drop(['drug_name', 'batch_number', 'expiry_date', 'approval_status'], axis=1)

In [3]:
df_encoded = pd.get_dummies(df, drop_first=True)


In [4]:
severity_map = {'Mild': 1, 'Moderate': 2, 'Severe': 3}
df['side_effect_severity_num'] = df['side_effect_severity'].map(severity_map)


In [5]:
df['num_side_effects'] = df['side_effects'].astype(str).apply(lambda x: len(x.split(',')))
df['risk_score'] = df['num_side_effects'] * df['side_effect_severity_num']


In [6]:
df = df.drop(['side_effects', 'side_effect_severity'], axis=1)
df_encoded = pd.get_dummies(df, drop_first=True)
X = df_encoded.drop('approval_status_encoded', axis=1)
y = df_encoded['approval_status_encoded']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Accuracy (Improved Random Forest):", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


Accuracy (Improved Random Forest): 0.3854166666666667
Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.29      0.32        93
           1       0.38      0.41      0.39        99
           2       0.42      0.45      0.43        96

    accuracy                           0.39       288
   macro avg       0.38      0.38      0.38       288
weighted avg       0.38      0.39      0.38       288

